In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import gc

In [3]:
train = pd.read_csv('./d_train_20180102.csv', encoding = 'EUC_CN')
test = pd.read_csv('./d_test_A_20180102.csv', encoding = 'EUC_CN')
sample = pd.read_csv('./d_sample_20180102.csv', encoding = 'EUC_CN')

In [4]:
train.shape

(5642, 42)

In [5]:
train.head(100)

,id,性别,年龄,体检日期,*天门冬氨酸氨基转换酶,*丙氨酸氨基转换酶,*碱性磷酸酶,*r-谷氨酰基转换酶,*总蛋白,白蛋白,...,血小板计数,血小板平均体积,血小板体积分布宽度,血小板比积,中性粒细胞%,淋巴细胞%,单核细胞%,嗜酸细胞%,嗜碱细胞%,血糖
0,1,男,41,12/10/2017,24.96,23.10,99.59,20.23,76.88,49.60,...,166.0,9.9,17.4,0.164,54.1,34.2,6.5,4.7,0.6,6.06
1,2,男,41,19/10/2017,24.57,36.25,67.21,79.00,79.43,47.76,...,277.0,9.2,10.3,0.260,52.0,36.7,5.8,4.7,0.8,5.39
2,3,男,46,26/10/2017,20.82,15.23,63.69,38.17,86.23,48.00,...,241.0,8.3,16.6,0.199,48.1,40.3,7.7,3.2,0.8,5.59
3,4,女,22,25/10/2017,14.99,10.59,74.08,20.22,70.98,44.02,...,252.0,10.3,10.8,0.260,41.7,46.5,6.7,4.6,0.5,4.30
4,5,女,48,26/10/2017,20.07,14.78,75.79,22.72,78.05,41.83,...,316.0,11.1,14.0,0.350,56.6,33.1,9.1,0.6,0.6,5.42
5,6,女,74,18/10/2017,23.72,22.59,81.23,23.35,76.46,45.85,...,249.0,8.5,17.0,0.211,42.9,47.0,7.1,2.1,1.0,5.97
6,7,男,31,21/09/2017,24.97,25.53,109.03,65.42,80.82,46.40,...,246.0,10.8,13.3,0.270,52.9,32.0,11.3,3.1,0.7,5.11
7,8,男,55,21/09/2017,37.32,40.03,88.49,25.15,74.17,41.63,...,282.0,10.5,13.0,0.300,52.8,36.9,6.6,2.8,0.9,5.94
8,9,男,39,16/10/2017,21.70,39.17,102.91,67.09,76.12,49.31,...,275.0,9.8,11.4,0.270,53.1,35.9,7.3,3.3,0.4,5.66
9,10,女,35,10/10/2017,18.89,15.79,78.21,25.75,75.94,46.64,...,247.0,10.7,12.6,0.270,65.6,27.3,5.4,1.0,0.7,5.48


In [6]:
x_train = train.drop(["血糖", "体检日期", "性别"], axis=1).values
x_test = test.drop(["体检日期", "性别"], axis=1).values
y_train = train["血糖"].values

In [7]:
split = 1250
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

In [8]:
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

print('Training ...')

params = {}
params['eta'] = 0.02
params['objective'] = 'reg:linear'
params['eval_metric'] = 'mae'
params['max_depth'] = 4
params['silent'] = 1

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=100, verbose_eval=100)

d_test = xgb.DMatrix(x_test)

p_test = clf.predict(d_test)
print(len(p_test))
sub = pd.DataFrame(p_test)

print('Writing csv ...')
sub.to_csv('xgb_starter.csv', index=False, float_format='%.4f', header=None) # Thanks to @inversion

Training ...
[0]	train-mae:4.93925	valid-mae:5.05689
Multiple eval metrics have been passed: 'valid-mae' will be used for early stopping.

Will train until valid-mae hasn't improved in 100 rounds.
[100]	train-mae:0.727925	valid-mae:0.878523
[200]	train-mae:0.474463	valid-mae:0.771651
Stopping. Best iteration:
[149]	train-mae:0.514483	valid-mae:0.759525

1000
Writing csv ...


In [9]:
import lightgbm as lgb

In [10]:
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.0021 # shrinkage_rate
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'          # or 'mae'
params['sub_feature'] = 0.345    # feature_fraction (small values => use very different submodels)
params['bagging_fraction'] = 0.85 # sub_row
params['bagging_freq'] = 40
params['num_leaves'] = 512        # num_leaf
params['min_data'] = 500         # min_data_in_leaf
params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
params['verbose'] = 0
params['feature_fraction_seed'] = 2
params['bagging_seed'] = 3

l_train = lgb.Dataset(x_train, label=y_train)
l_valid = lgb.Dataset(x_valid, label=y_valid)
l_test = lgb.Dataset(x_test)

In [11]:
clf = lgb.train(params, l_train, 10000, valid_sets=l_valid, early_stopping_rounds=100, verbose_eval=100)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 0.803295
[200]	valid_0's l1: 0.799129
[300]	valid_0's l1: 0.794726
[400]	valid_0's l1: 0.790987
[500]	valid_0's l1: 0.788532
[600]	valid_0's l1: 0.786198
[700]	valid_0's l1: 0.783117
[800]	valid_0's l1: 0.781244
[900]	valid_0's l1: 0.779379
[1000]	valid_0's l1: 0.777347
[1100]	valid_0's l1: 0.776043
[1200]	valid_0's l1: 0.775583
[1300]	valid_0's l1: 0.775311
[1400]	valid_0's l1: 0.77436
[1500]	valid_0's l1: 0.773979
[1600]	valid_0's l1: 0.773783
[1700]	valid_0's l1: 0.77369
[1800]	valid_0's l1: 0.772879
Early stopping, best iteration is:
[1760]	valid_0's l1: 0.772609


In [12]:
p_test = clf.predict(x_test)
print(len(p_test))
sub = pd.DataFrame(p_test)

print('Writing csv ...')
sub.to_csv('lgb_starter.csv', index=False, float_format='%.4f', header=None) # Thanks to @inversion

1000
Writing csv ...


In [14]:
# ensemble
x_r = pd.read_csv('xgb_starter.csv', header=None)
l_r = pd.read_csv('lgb_starter.csv', header=None)


sub = pd.DataFrame(x_r.values + l_r.values) / 2
print(len(sub))

1000


In [15]:
sub.to_csv('ensemble.csv', index=False, float_format='%.4f', header=None) # Thanks to @inversion